# 一、依序下载的脚本

In [2]:
import time
from pathlib import Path
from typing import Callable

import httpx

In [10]:
POP20_CC = ('CH IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()
BASE_URL = 'http://mp.ituring.com.cn/files/flags'
DEST_DIR = Path('downloaded')

In [11]:
def save_flag(img: bytes, filename: str) -> None:
    (DEST_DIR / filename).write_bytes(img)

In [12]:
def get_flag(cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = httpx.get(url, timeout=6.1,
                     follow_redirects=True)
    resp.raise_for_status()  # 避免悄无声息的失败
    return resp.content


In [13]:
def download_many(cc_list: list[str]) -> int:
    for cc in sorted(cc_list):
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True)
    return len(cc_list)

In [14]:
def main(downloader: Callable[[list[str]], int]) -> None:
    DEST_DIR.mkdir(exist_ok=True)
    t0 = time.perf_counter()
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

In [15]:
main(downloader=download_many)

BD BR CD CH DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 5.63s


# 二、使用concurrent.futures模块下载

In [16]:
from concurrent import futures

def download_one(cc: str):  # 职程
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

def download_many(cc_list: list[str]) -> int:
    with futures.ThreadPoolExecutor() as executor:  # 把依序执行的for循环改成函数，并发调用
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

In [18]:
main(downloader=download_many)

BR DE BD CH CD EG NG ET MX FR ID JP IN IR PH PK VN TR US RU 
20 downloads in 1.26s


In [20]:
# 另一种复杂的download_many
def download_many(cc_list: list[str]) -> int:
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do: list[futures.Future] = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            print(f'Scheduled for {cc}: {future}')
        for count, future in enumerate(futures.as_completed(to_do), 1):
            res: str = future.result()
            print(f'{future} result: {res!r}')
    return count

In [21]:
main(downloader=download_many)

Scheduled for BR: <Future at 0x20e9ea2b5f0 state=running>
Scheduled for CH: <Future at 0x20e9ea2b4d0 state=running>
Scheduled for ID: <Future at 0x20e9ea2a9c0 state=running>
Scheduled for IN: <Future at 0x20e9ea29c10 state=pending>
Scheduled for US: <Future at 0x20e9ea2a2a0 state=pending>
ID CH <Future at 0x20e9ea2a9c0 state=finished returned str> result: 'ID'
<Future at 0x20e9ea2b4d0 state=finished returned str> result: 'CH'
BR <Future at 0x20e9ea2b5f0 state=finished returned str> result: 'BR'
IN <Future at 0x20e9ea29c10 state=finished returned str> result: 'IN'
US <Future at 0x20e9ea2a2a0 state=finished returned str> result: 'US'

5 downloads in 0.80s


# 三、基于进程的方案

In [ ]:
def download_many(cc_list: list[str]) -> int:
    with futures.ProcessPoolExecutor() as executor:  # 把依序执行的for循环改成函数，并发调用
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))